In [1]:
import time
from selenium import webdriver
from pprint import pprint
import concurrent
import json
import pandas as pd
import random

In [2]:
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
# we dont need to load images
chrome_prefs = {
      'profile.managed_default_content_settings.images': 2,
    }
# chrome_options.add_argument('user-agent="Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.108 Mobile Safari/537.36"')
chrome_options.add_experimental_option('prefs', chrome_prefs)
def get_driver():
    return webdriver.Chrome(options=chrome_options)

In [3]:
# driver = get_driver()

In [4]:
def scrolltobottom(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
def scrolltotop(driver):
    driver.execute_script("window.scrollTo(0, 0)")
def scroll(driver, count):
     for i in range(count):
        time.sleep(.5)
        scrolltotop(driver)
        scrolltobottom(driver)
def get_pkg(url):
    return url.replace('https://play.google.com/store/apps/details?id=','')

In [5]:
def appendmsg(msg, filename):
    with open(filename, 'ab') as f:
        f.write(msg.encode('utf-8'))
        f.write('\n'.encode('utf-8'))
        
BUFSIZ = 1
BUFFER = []
def log(appid, l):
    BUFFER.append((appid, l))
    if len(BUFFER) >= BUFSIZ:
        for i in range(BUFSIZ):
            appid, name = BUFFER.pop(0)
            item = {appid: l}
            appendmsg(json.dumps(item, ensure_ascii=False), 'similar_raw.jl')

In [6]:
df = pd.read_csv('data_essential.csv')
df

,name,pkg,publisher,genres,content_ratings,is_editor_choice,price,inapp_details,has_video,num_screenshot,...,rating_2,rating_1,rating_count,update_time,size,min_install,version,android_version,interactive_elements,in_app_products
0,WhatsApp Messenger,com.whatsapp,WhatsApp Inc.,Communication,Everyone,False,0.00,NaN,False,6,...,2118908,5344252,87499059,2019-04-24,Varies with device,1000000000,Varies with device,Varies with device,"Users Interact, Digital Purchases",NaN
1,Facebook,com.facebook.katana,Facebook,Social,Teen,False,0.00,Contains Ads·Offers in-app purchases,False,5,...,3499060,10113178,86433210,2019-04-25,Varies with device,1000000000,Varies with device,Varies with device,"Users Interact, Shares Info, Shares Location, ...",$0.99 - $399.99 per item
2,Instagram,com.instagram.android,Instagram,Social,Teen,True,0.00,Contains Ads,False,5,...,1448081,3356089,80808062,2019-04-23,Varies with device,1000000000,Varies with device,Varies with device,"Users Interact, Shares Info, Shares Location",NaN
3,Messenger – Text and Video Chat for Free,com.facebook.orca,Facebook,Communication,Everyone,True,0.00,Offers in-app purchases,False,8,...,2502559,7830646,65957119,2019-04-23,Varies with device,1000000000,Varies with device,Varies with device,"Users Interact, Shares Location, Digital Purch...",$0.99 - $399.99 per item
4,Clash of Clans,com.supercell.clashofclans,Supercell,Strategy,Everyone 10+,True,0.00,Offers in-app purchases,True,21,...,737549,2093231,48650861,2019-04-23,103M,500000000,11.446.24,4.1 and up,"Users Interact, Digital Purchases",$0.99 - $100.00 per item
5,"Clean Master - Antivirus, Applock & Cleaner",com.cleanmaster.mguard,Cheetah Mobile,Tools,Everyone,False,0.00,Contains Ads·Offers in-app purchases,True,6,...,519694,975487,44229745,2019-04-22,20M,1000000000,7.0.8,Varies with device,NaN,$2.99 - $10.68 per item
6,YouTube,com.google.android.youtube,Google LLC,Video Players & Editors,Teen,True,0.00,Contains Ads,False,5,...,1092518,3363947,43487619,2019-04-19,Varies with device,5000000000,Varies with device,Varies with device,"Users Interact, Digital Purchases",NaN
7,Subway Surfers,com.kiloo.subwaysurf,Kiloo,Arcade,Everyone 10+,True,0.00,Contains Ads·Offers in-app purchases,True,25,...,695792,1516050,29993489,2019-04-25,88M,1000000000,1.102.0,4.1 and up,Digital Purchases,$0.99 - $99.99 per item
8,Clash Royale,com.supercell.clashroyale,Supercell,Strategy,Everyone 10+,True,0.00,Offers in-app purchases,True,19,...,331601,1490455,25698937,2019-04-15,87M,100000000,2.7.1,4.1 and up,"Users Interact, Digital Purchases",$0.99 - $99.99 per item
9,"Security Master - Antivirus, VPN, AppLock, Boo...",com.cleanmaster.security,Cheetah Mobile (AppLock & AntiVirus),Tools,Everyone,False,0.00,Contains Ads·Offers in-app purchases,False,7,...,296582,531227,25579290,2019-04-22,22M,500000000,4.9.3,Varies with device,Digital Purchases,$0.99 - $35.99 per item


In [7]:
SEARCHED = set()
with open('similar_raw.jl', encoding='utf-8') as f:
    for l in f.readlines():
        try:
            j = json.loads(l)
            for appid in j:
                SEARCHED.add(appid)
        except:
            pass
print(f'there are {len(SEARCHED)} searched apps')

there are 2053 searched apps


In [8]:
QUEUE = []
for i in range(len(df)):
    if df['pkg'][i] not in SEARCHED:
        QUEUE.append(df['pkg'][i])
print(f'there are {len(QUEUE)} in queue')
QUEUE[:10]

there are 640691 in queue


['com.estrongs.android.pop',
 'com.duapps.recorder',
 'com.utorrent.client',
 'com.ForgeGames.SpecialForcesGroup2',
 'photo.video.instasaveapp',
 'air.com.eastsidegamestudio.PFGrassRoots',
 'com.myboyfriendisageek.videocatcher.demo',
 'com.gipnetix.escapeaction',
 'com.jamendoandoutly.mainpakage',
 'com.popularapp.videodownloaderforinstagram']

In [9]:
BASEURL = 'https://play.google.com/store/apps/details?id={}'
def get_similar_pkgs(driver, pkg):
    driver.get(BASEURL.format(pkg))
    
    buttons = driver.find_elements_by_class_name('xwY9Zc')
    button = None
    for b in buttons:
        if 'similar' in b.text.lower():
            button = b
            break
    
    if button == None:
        log(pkg, [])
        appendmsg(pkg, 'searched_similar.txt')
        SEARCHED.add(pkg)
        return []
    button.click()
    
    # we're now at the similar apps page, scroll a few times
    scroll(driver, 5)
    
    # get all the cards wXUyZd
    cards = driver.find_elements_by_class_name('wXUyZd')
    l = [
        get_pkg(card.find_element_by_tag_name('a').get_attribute('href')) \
        for card in cards
    ]
    
    log(pkg, l)
    appendmsg(pkg, 'searched_similar.txt')
    SEARCHED.add(pkg)
    
    return l

In [10]:
def task(driver):
    while True:
        pkg = QUEUE.pop(0)
        get_similar_pkgs(driver, pkg)

In [11]:
from concurrent.futures import ThreadPoolExecutor
NUMTHREADS = 24

executor = ThreadPoolExecutor(max_workers=NUMTHREADS)
for i in range(NUMTHREADS):
    executor.submit(task, get_driver())
    time.sleep(1)

In [30]:
print(len(QUEUE))
QUEUE[:10]

640460


['com.icloudzone.DeathMoto2',
 'com.aufeminin.marmiton.activities',
 'com.marcow.birthdaylist',
 'com.shinycore.picsayfree',
 'io.voodoo.dune',
 'com.atomgame.gunwar',
 'com.fast.free.unblock.thunder.vpn',
 'ch.bitspin.timely',
 'com.sonyericsson.music',
 'com.busuu.android.enc']

In [13]:
asfdasdfasdf

NameError: name 'asfdasdfasdf' is not defined

In [ ]:
dd = get_driver()

In [ ]:
get_similar_pkgs(dd, QUEUE.pop(0))